## SQLAlchemy Expression Language

SQA EL - представляет собой систему представления структур и выражений реляционных баз данных с использованием конструкций Python. Эти конструкции похожи на соответствующие структуры, при этом дают некоторую абстракцию для работы с различными бэкэндами.

Язык выражений представляет метод написания SQL-выражений, нейтральных к серверной части, но при этом с возможностью использовать фишек соответствующего бэка.

EL отличается от API ORM, который основан на EL. ORM - более верхнеуровневый способ работы с БД и фактически является приложением EL, в то время как EL - это система примитивных конструкций.

### Создание таблиц

In [2]:
import sqlalchemy
from sqlalchemy import create_engine

Для работы используем in-memory SQLite DB. 

In [5]:
# engine - это ключевой интерфейс к БД, адаптированный в соответствии с диалектом
engine = create_engine(
    'sqlite:///:memory:', 
    echo=True # позволяет видеть все SQL запросы по ходу выполнения
)

EL часто подразумевает работу с колонками таблиц. Набор таблиц и их дочерних элементов называется метаданными базы данных. 

Сейчас мы будем создавать таблицы вручную, но SQA может "импортировать" наборы таблиц из существующей БД при помощи механизма reflection.

In [6]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
metadata_obj = MetaData() # метаданные БД
# описываем 2 объекта, соответсвующие будущим таблицам
users = Table(
  'users', 
  metadata_obj,
  Column('id', Integer, primary_key=True),
  Column('name', String),
  Column('fullname', String),
)

addresses = Table('addresses', 
  metadata_obj,
  Column('id', Integer, primary_key=True),
  Column('user_id', None, ForeignKey('users.id')),
  Column('email_address', String, nullable=False)
)
# создаем таблицы
metadata_obj.create_all(engine)

2022-07-22 14:18:54,432 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-22 14:18:54,433 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-07-22 14:18:54,434 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-22 14:18:54,435 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2022-07-22 14:18:54,436 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-22 14:18:54,437 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("addresses")
2022-07-22 14:18:54,438 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-22 14:18:54,440 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("addresses")
2022-07-22 14:18:54,441 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-22 14:18:54,442 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-07-22 14:18:54,443 INFO sqlalchemy.engine.Engine [no key 0.00103s] ()
2022-07-22 14:18:54,463 INFO sqlalchemy.engine.Engine 
CREATE TABLE addresses

Для добавления записи создаем statement. По умолчанию мы должны предоставить все столбцы, но это можно настроить.

In [7]:
ins_stmt = users.insert()
print(ins_stmt)

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)


In [9]:
data = {'name': 'jack', 'fullname': 'Jack Jones'}
ins_stmt = users.insert().values(**data)
# данные не отображаются в запросе сейчас, но хранятся в statement
print(ins_stmt)

INSERT INTO users (name, fullname) VALUES (:name, :fullname)


Для выполнения команды нужно создать объект подключения.

In [12]:
conn = engine.connect() # объект-подключение
result = conn.execute(ins_stmt) # курсор, содержащий информацию о работе запроса
result.inserted_primary_key

2022-07-22 14:25:33,077 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2022-07-22 14:25:33,079 INFO sqlalchemy.engine.Engine [cached since 78.52s ago] ('jack', 'Jack Jones')
2022-07-22 14:25:33,080 INFO sqlalchemy.engine.Engine COMMIT


(2,)

In [13]:
# еще один способ добавления записи
data = {'id': 3, 'name': 'wendy', 'fullname': 'Wendy Will'}
conn.execute(
    users.insert(),
    data
)

2022-07-22 14:27:10,822 INFO sqlalchemy.engine.Engine INSERT INTO users (id, name, fullname) VALUES (?, ?, ?)
2022-07-22 14:27:10,823 INFO sqlalchemy.engine.Engine [generated in 0.00151s] (3, 'wendy', 'Wendy Will')
2022-07-22 14:27:10,825 INFO sqlalchemy.engine.Engine COMMIT


In [14]:
# добавление множества записей
# все словари должны содержать одинаковые ключи
data = [
   {'user_id': 1, 'email_address' : 'jack@yahoo.com'},
   {'user_id': 1, 'email_address' : 'jack@msn.com'},
   {'user_id': 2, 'email_address' : 'www@www.org'},
   {'user_id': 2, 'email_address' : 'wendy@aol.com'},
]

conn.execute(
    addresses.insert(),
    data
)

2022-07-22 14:28:02,030 INFO sqlalchemy.engine.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2022-07-22 14:28:02,032 INFO sqlalchemy.engine.Engine [generated in 0.00155s] ((1, 'jack@yahoo.com'), (1, 'jack@msn.com'), (2, 'www@www.org'), (2, 'wendy@aol.com'))
2022-07-22 14:28:02,034 INFO sqlalchemy.engine.Engine COMMIT


### Выборки

Основной конструкцией, используемой для генерации операторов SELECT, является функция select()

In [35]:
from sqlalchemy.sql import select
cursor = conn.execute(
    select(users),
)
result = list(cursor) 
# после того, как курсор исчерпан, он автоматически закроется
# но это можно сделать и явно
cursor.close()
# по умолчанию возвращает строчки в виде похожих на namedtuple объектоа
print(f'{result=}')
print(f'{result[0].id=}')
print(f'{dict(result[0])=}')
# можно получить строчки в виде словарей при помощи поля ._mapping
result_dicts =[r._mapping for r in result]
print(f'{result_dicts=}')

2022-07-22 20:12:47,700 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname 
FROM users
2022-07-22 20:12:47,701 INFO sqlalchemy.engine.Engine [cached since 2.058e+04s ago] ()
result=[(1, 'jack', 'Jack Jones'), (2, 'jack', 'Jack Jones'), (3, 'wendy', 'Wendy Will')]
result[0].id=1
dict(result[0])={'id': 1, 'name': 'jack', 'fullname': 'Jack Jones'}
result_dicts=[{'id': 1, 'name': 'jack', 'fullname': 'Jack Jones'}, {'id': 2, 'name': 'jack', 'fullname': 'Jack Jones'}, {'id': 3, 'name': 'wendy', 'fullname': 'Wendy Will'}]


In [28]:
row = result_dicts[0]
# можно обращаться по названию поля
print(f"{row['id']=}")
# можно обращаться, используя объект Column
print(f"{row[users.c.id]=}")

row['id']=1
row[users.c.id]=1


В функцию `select` можно передавать список столбцов, причем из разных таблиц (или список таблиц). Нужно быть очень аккуратным, т.к. перечисление столбцов из разных таблиц без доп. условий фактически означает декартово произведение строк.

In [36]:
conn.execute(
    select(users, addresses)
)

2022-07-22 20:52:38,754 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname, addresses.id AS id_1, addresses.user_id, addresses.email_address 
FROM users, addresses
2022-07-22 20:52:38,756 INFO sqlalchemy.engine.Engine [generated in 0.00147s] ()


C:\Users\Nikita\AppData\Local\Temp\ipykernel_3824\1968581075.py:1: SAWarning: SELECT statement has a cartesian product between FROM element(s) "users" and FROM element "addresses".  Apply join condition(s) between each element to resolve.
  conn.execute(


Чтобы получить корректное объединение, нужно добавить условие WHERE.

In [37]:
conn.execute(
    select(users, addresses).where(users.c.id == addresses.c.user_id)
)

2022-07-22 20:53:39,687 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname, addresses.id AS id_1, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2022-07-22 20:53:39,689 INFO sqlalchemy.engine.Engine [generated in 0.00140s] ()


Мы можем давать алиасы столбцам при помощи метода `label`

In [55]:
list(
    map(
        dict, 
        conn.execute(
            select(
                (users.c.fullname).label('title')
            )
        )
    )
)

2022-07-22 21:07:06,642 INFO sqlalchemy.engine.Engine SELECT users.fullname AS title 
FROM users
2022-07-22 21:07:06,643 INFO sqlalchemy.engine.Engine [cached since 27.5s ago] ()


[{'title': 'Jack Jones'}, {'title': 'Jack Jones'}, {'title': 'Wendy Will'}]

SQA позволяет создавать выражения при помощи обычных операторов.
* операция над двумя столбцами порождает BinaryExpression
* операция над столбцом и литералов порождает выражение с параметров
* можно явно описывать операторы при помощи метода `.op`

Стандартные операторы:
* ==
* !=
* like (и регистронезависимая версия ilike)
* in_
* not_in
* is_
* is_not
* and_ (или оператор &, или просто перечислить условия в `where` через запятую)
* or_ (или оператор |)
* not_ (или оператор ~)

In [42]:
exp1 = users.c.id == addresses.c.user_id
print(exp1, type(exp1))

users.id = addresses.user_id <class 'sqlalchemy.sql.elements.BinaryExpression'>


In [44]:
print(users.c.id == 7)
print(users.c.id == None)
print(users.c.id.op("<>")(addresses.c.user_id))

users.id = :id_1
users.id IS NULL
users.id <> addresses.user_id


In [51]:
from sqlalchemy import and_, or_, not_
print(
    and_(
        users.c.id == 7, 
        users.c.name.like('j%'),
        or_(
                addresses.c.email_address == 'wendy@aol.com',
                addresses.c.email_address == 'jack@yahoo.com'
        ),
    )
)

users.id = :id_1 AND users.name LIKE :name_1 AND (addresses.email_address = :email_address_1 OR addresses.email_address = :email_address_2)


In [50]:
from sqlalchemy import and_, or_, not_
print(
    and_(
        (users.c.id == 7) & 
        (users.c.name.like('j%')) & 
        (
            (addresses.c.email_address == 'wendy@aol.com')  | 
            (addresses.c.email_address == 'jack@yahoo.com')
        ),
    )
)

users.id = :id_1 AND users.name LIKE :name_1 AND (addresses.email_address = :email_address_1 OR addresses.email_address = :email_address_2)


При необходимости можно выполнить запрос на языке SQL

In [58]:
from sqlalchemy.sql import text
s = text(
    '''SELECT users.fullname || ' -> ' || users.name AS title
       FROM users
       WHERE users.name BETWEEN :x AND :y'''
)
conn.execute(s, {"x": "m", "y": "z"}).fetchall()


2022-07-22 21:09:58,902 INFO sqlalchemy.engine.Engine SELECT users.fullname || ' -> ' || users.name AS title
       FROM users
       WHERE users.name BETWEEN ? AND ?
2022-07-22 21:09:58,903 INFO sqlalchemy.engine.Engine [generated in 0.00164s] ('m', 'z')


[('Wendy Will -> wendy',)]

При написании запросов мы можем столкнуться с проблемой, когда несколько колонок в результирующей таблице называются одинаково. Чтобы избежать проблем, мы можем явно указать, каким таблицам принадлежат столбцы.

In [62]:
stmt = text("SELECT users.id, addresses.id, users.id, "
    "users.name, addresses.email_address AS email "
    "FROM users JOIN addresses ON users.id=addresses.user_id "
    "WHERE users.id = 1").columns(
       users.c.id,
       addresses.c.id,
       addresses.c.user_id,
       users.c.name,
       addresses.c.email_address
    )
result = conn.execute(stmt)
row = result.fetchone()
print(f'{row[users.c.id]=}')

2022-07-22 21:14:18,451 INFO sqlalchemy.engine.Engine SELECT users.id, addresses.id, users.id, users.name, addresses.email_address AS email FROM users JOIN addresses ON users.id=addresses.user_id WHERE users.id = 1
2022-07-22 21:14:18,452 INFO sqlalchemy.engine.Engine [cached since 61.03s ago] ()
row[users.c.id]=1


Мы можем комбинировать программный и текстовый подходы к созданию запросов

In [64]:
print(
    and_(
        text("users.id == 7"), 
        text("users.name LIKE 'j%'"),
        or_(
                text("addresses.email_address == 'wendy@aol.com'"),
                text("addresses.email_address == 'jack@yahoo.com'")
        ),
    )
)

users.id == 7 AND users.name LIKE 'j%' AND (addresses.email_address == 'wendy@aol.com' OR addresses.email_address == 'jack@yahoo.com')


### Запросы с модификаторами

Мы можем писать запросы с:
* группировками (group_by + func) и фильтрами на них (having)
* сортировками (order_by)
* алиасами (.alias)
* подзапросами (subquery)
* ограничение на кол-во строк (limit) 
* сдвиг (offset)

In [66]:
from sqlalchemy import func, asc, desc
stmt = select(
        addresses.c.user_id,
        func.count(addresses.c.id).label('num_addresses')
).group_by("user_id").order_by(
        asc("user_id"), 
        desc("num_addresses")
)

conn.execute(stmt).fetchall()


2022-07-22 21:23:54,809 INFO sqlalchemy.engine.Engine SELECT addresses.user_id, count(addresses.id) AS num_addresses 
FROM addresses GROUP BY addresses.user_id ORDER BY addresses.user_id ASC, num_addresses DESC
2022-07-22 21:23:54,810 INFO sqlalchemy.engine.Engine [generated in 0.00117s] ()


[(1, 2), (2, 2)]

In [89]:
from sqlalchemy import func, asc, desc
print(
    select(
        addresses.c.user_id,
        func.count(addresses.c.id).label('num_addresses')
    ).group_by(addresses.c.user_id).having(
            func.length(addresses.c.user_id) > 2
    )
)



SELECT addresses.user_id, count(addresses.id) AS num_addresses 
FROM addresses GROUP BY addresses.user_id 
HAVING length(addresses.user_id) > :length_1


In [70]:
a1 = addresses.alias("a")
stmt = select(a1).where(a1.c.id > 5)
print(stmt)

SELECT a.id, a.user_id, a.email_address 
FROM addresses AS a 
WHERE a.id > :id_1


In [72]:
subq = stmt.subquery("sq")
stmt2 = select(subq)
print(stmt2)

SELECT sq.id, sq.user_id, sq.email_address 
FROM (SELECT a.id AS id, a.user_id AS user_id, a.email_address AS email_address 
FROM addresses AS a 
WHERE a.id > :id_1) AS sq


### Объединения

Мы уже выполняли объединения в наших примерах, просто помещая две таблицы либо в предложение select. Но если мы хотим создать реальную конструкцию “JOIN” или “OUTERJOIN”, мы используем методы FromClause.join() и FromClause.outerjoin(), доступ к которым чаще всего осуществляется из левой таблицы в соединении.

In [73]:
# SQA автоматически поняла, по каким полям объединить таблицы на основе внешнего 
# ключа из описания таблицы
print(
    users.join(addresses)
)

users JOIN addresses ON users.id = addresses.user_id


In [75]:
# мы можем явно указать правило для объединения
print(
    users.join(
        addresses, 
        users.c.id > addresses.c.user_id
    )
)

users JOIN addresses ON users.id > addresses.user_id


In [76]:
# для использования объединения таблиц в разделе FROM мы используем
# select_from
print(
    select(users.c.id).select_from(
        users.join(
            addresses, 
            users.c.id > addresses.c.user_id
        )
    )
)

SELECT users.id 
FROM users JOIN addresses ON users.id > addresses.user_id


## Common table expressions

CTE представляет собой временный именованный результирующий набор, на который можно ссылаться в инструкции SELECT, INSERT, UPDATE или DELETE. CTE также можно использовать в представлении. Конструкция CTE - отличный способ предоставить источник строк, семантически аналогичный использованию подзапроса, но с гораздо более простым форматом, где источник строк аккуратно спрятан в верхней части запроса, и на него можно ссылаться в любом месте основного оператора, как на обычную таблицу.

SQLAlchemy поддерживает эту конструкцию через объект CTE, который обычно получают с помощью метода Select.cte() для конструкции Select.

In [78]:
cte_s = select(users.c.id).select_from(
    users.join(
        addresses, 
        users.c.id > addresses.c.user_id
    )
).cte("my_cte")
print(
    select(cte_s)
)

WITH my_cte AS 
(SELECT users.id AS id 
FROM users JOIN addresses ON users.id > addresses.user_id)
 SELECT my_cte.id 
FROM my_cte


## Другие возможности EL

Мы можем именовать параметры запроса при помощи bindparam. Один и тот же параметр можно использовать несколько раз в запросе (достаточно указывать одинаковое имя)

In [81]:
from sqlalchemy.sql import bindparam
s = users.select().where(users.c.name == bindparam('username', type_=String))
conn.execute(s, {"username": "wendy"}).fetchall()


2022-07-22 21:53:04,915 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.name = ?
2022-07-22 21:53:04,917 INFO sqlalchemy.engine.Engine [cached since 40.52s ago] ('wendy',)


[(3, 'wendy', 'Wendy Will')]

Можно использовать существующие SQL-функции, которые лежат в пакете func, в том числе оконные функции

In [82]:
print(
    select(
        func.max(addresses.c.email_address, type_=String).label('maxemail')
    )
)

SELECT max(addresses.email_address) AS maxemail 
FROM addresses


In [84]:
print(
    select(
        users.c.id,
        func.row_number().over(order_by=users.c.name).label("num")
    )
)

SELECT users.id, row_number() OVER (ORDER BY users.name) AS num 
FROM users


Доступны преобразования типов при помощи функции cast. Также доступна похожая функция type_coerce, которая отличается тем, что не добавляет выражение CAST в запрос.

In [86]:
from sqlalchemy import cast

print(
    select(
        cast(
            users.c.id, String
        ).label("sid")
    )
)

SELECT CAST(users.id AS VARCHAR) AS sid 
FROM users


Доступны теоретико-множественные операции над результирующими множествами:
* union (union_all)
* intersect (intersect_all)
* except (except_all)

In [87]:
from sqlalchemy import union
print(
    union(
        select(users.c.id).where(users.c.id == 1),
        select(users.c.id).where(users.c.id == 2),

    )
)

SELECT users.id 
FROM users 
WHERE users.id = :id_1 UNION SELECT users.id 
FROM users 
WHERE users.id = :id_2


## Обновление таблиц

Для обновления используем метод `update`

In [92]:
print(
    users.update().where(users.c.name == 'a').values(name='b')
)

UPDATE users SET name=:name WHERE users.name = :name_1


In [93]:
# можем использовать параметры
print(
    users.update().where(users.c.name == bindparam('old')).values(name=bindparam('new'))
)
conn.execute(
    users.update().where(users.c.name == bindparam('old')).values(name=bindparam('new')),
    [
        {'oldname':'jack', 'newname':'ed'},
        {'oldname':'wendy', 'newname':'mary'},
    ]
)

UPDATE users SET name=:new WHERE users.name = :old


## Удаление таблиц

Для удаления используем метод `delete`

In [94]:
print(
    addresses.delete()
)

DELETE FROM addresses


In [96]:
print(
    addresses.delete().where(addresses.c.id > 5)
)

DELETE FROM addresses WHERE addresses.id > :id_1
